In [1]:
# Project Overview: Create Relevant Spotify Playlist with Python
# Pre-work: Set up your Account & Register your application @ https://developer.spotify.com/documentation/web-api/quick-start/
# pip install requirements.txt
# 0. Declare name of playlist to be created
# 1. Import Packages and establish creds to connect to Spotify
# 2. Get raw Spotify data of Liked (Hearted) songs into a dataframe
# 3. Add audio features
# 4. Add genre data
# 5. Create genre score
### Repeat process (steps 2-5) for disliked songs (based on dislike playlist) ###
# 6. Create playlist df (to access disliked playlist)
# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
# 9. Get raw data from the songs
# ....
# 10. Create playlist

In [2]:
# 0. Declare name of playlist to be created
pred_like_playlist_name = "test789"
genre_threshold = 21

In [1]:
#### ALL FUNCTIONS BELOW ####

# 1. Import Packages and establish creds to connect to Spotify
import sys
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
import requests
import math
import time
import pandas as pd
from IPython.display import display
from pandas.io.json import json_normalize
import numpy as np 
#import matplotlib.pyplot as plt  
import inspect
import pickle
from datetime import datetime, timedelta

def define_scope():
    global username, scope, token, sp, user, headers
    username = os.environ['SPOTIPY_USERNAME']
    client_id = os.environ['SPOTIPY_CLIENT_ID']
    client_secret = os.environ['SPOTIPY_CLIENT_SECRET']
    redirect_uri = 'http://google.com'
    scope = '''
    playlist-modify-private 
    playlist-modify-public 
    playlist-read-collaborative 
    playlist-read-private
    user-follow-modify 
    user-follow-read 
    user-library-modify 
    user-library-read 
    user-modify-playback-state 
    user-read-currently-playing 
    user-read-email 
    user-read-playback-state 
    user-read-private 
    user-read-recently-played 
    user-top-read'''
    try:
        token = util.prompt_for_user_token(username=username, 
                                           scope=scope, 
                                           client_id=client_id, 
                                           client_secret=client_secret, 
                                           redirect_uri=redirect_uri,
                                           cache_path=f".cache-{username}")
    except (AttributeError, JSONDecodeError):
        os.remove(f".cache-{username}")
        token = util.prompt_for_user_token(username=username, 
                                           scope=scope, 
                                           client_id=client_id, 
                                           client_secret=client_secret, 
                                           redirect_uri=redirect_uri,
                                           cache_path=f".cache-{username}")
    sp = spotipy.Spotify(auth=token)
    user = sp.current_user()
    headers = {'Authorization': "Bearer {}".format(token)}
    return

# 2. Get raw Spotify data of Liked (i.e. Hearted) songs into a dataframe
# 2a. Get raw data of liked songs into a list
def req(url,headers):
    r = requests.get(url, headers=headers)
    return json.loads(r.text)

def get_raw_liked_song_list():
    print('Get_raw_liked_song_list')
    offset = 0
    url = f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}"

    liked_song_count = req(url,headers=headers)["total"]
    print("Liked song count: ", liked_song_count)

    raw_liked_songs = []
    for i in range(int(math.ceil(liked_song_count/50.0))):
        offset = 50*i
        raw_liked_songs.extend(req(f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}",headers=headers)["items"])

    print(f">> raw liked songs['0'] \n>> {raw_liked_songs[0]}")
    return raw_liked_songs

# 2b. transform liked song raw data list into df
# parse out data from Liked Songs' raw data
def song_metadata_to_df(raw_songs):
    print('Transforming song raw data list into df\n')
    define_scope()
    # create empty df
    df = pd.DataFrame(columns = [
        'album', 
        'artist_uri', 
        'artist',
        'track',
        'url',
        'popularity',
        'added_at',
        'release_date',
        'release_year',
        'uri'
        ])
    # parse out songs 100 at a time
    for i in range(0, len(raw_songs)):
        if (i/100).is_integer():
            print(f"{i}/{len(raw_songs)} parsed")
        try:
            album = raw_songs[i]['track']['album']['name']
            artist_uri = raw_songs[i]['track']['artists'][0]['uri'].split(':')[2]
            artist = raw_songs[i]['track']['album']['artists'][0]['name']
            track = raw_songs[i]['track']['name']
            url = raw_songs[i]['track']['external_urls']['spotify']
            popularity = raw_songs[i]['track']['popularity']
            added_at = raw_songs[i]['added_at']
            release_date = raw_songs[i]['track']['album']['release_date']
            release_year = release_date.split('-')[0]
            uri = raw_songs[i]['track']['uri']
            df = df.append(pd.DataFrame([[
                album, 
                artist_uri,
                artist, 
                track, 
                url, 
                popularity, 
                added_at, 
                release_date, 
                release_year,
                uri
                ]], columns = [
                'album', 
                'artist_uri', 
                'artist',
                'track',
                'url',
                'popularity',
                'added_at',
                'release_date',
                'release_year',
                'uri'
                ]))
        except:
            pass
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

#3. Add audio features 
# Combine liked song df with audio feature df
def add_audio_feats(df):
    print('Combining song_df with audio_feature_df')
    # get list of song uris
    song_uri_list = df.index.to_list()
    audio_feature_df = pd.DataFrame()
    for i in range(0,math.ceil(len(song_uri_list)/100)):
        define_scope()
        features = sp.audio_features(song_uri_list[(i)*100:100*(i+1)])
        audio_feature_df = audio_feature_df.append(pd.DataFrame(features))
    audio_feature_df.set_index('uri', inplace=True)
    # merge this 
    liked_song_df = df.merge(audio_feature_df, on = 'uri')
    display(liked_song_df.head())
    return liked_song_df.drop_duplicates()

#4. Add genre data
# 4a. Get artist_uris -> get genres
def get_genres(df):
    print('Getting genres\n')
    g = []
    artist_uris = df['artist_uri'].unique()
    print(len(artist_uris), ' distinct artists')
    for i in range(0, len(artist_uris)):
        if (i/100).is_integer():
            print(f"{i}/{len(artist_uris)} parsed")
        try:
            g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])
        except:
            define_scope()
            try:
                g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])
            except:
                pass
    # genres_df's artist_uri column stores a list of genres per artist uri
    genres_df = pd.DataFrame({'artist_uri': artist_uris, 'genre list': g})
    
    print('\ngenre_df')
    display(genres_df.head())
    return genres_df

# There is a list of genres for one artist that must be broken out
def explode_genres(df):
    genre_exploded_df = df.explode('genre list')
    
    print('\ngenre_exploded_df')
    display(genre_exploded_df.head())
    return genre_exploded_df

# Add these genres back to the df for each song
def add_genres(df, genre_exploded_df):
    # the following does not work oddly enough
    # liked_song_df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri', inplace = True)
    df = df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri')
    df.rename(columns={'genre list': 'genre'}, inplace=True)
    
    print('Merging genres into song_df')
    display(df.head())
    return df

#5. Create Genre Score

# Get genre counts for liked songs - the higher the count, the higher the score
def get_genre_counts(genre_exploded_df):
    print('Getting genre counts')
    # establish genre counts (which will determine score)
    genre_count_df = pd.DataFrame(genre_exploded_df['genre list'].value_counts()).reset_index()
    genre_count_df.columns = ['genre','genre count']
    print('\ngenre_count_df.head()')
    display(genre_count_df.head())
    return genre_count_df

# Merge the genre counts with the song df and sum the genre column
def establish_genre_score(genre_count_df, df):
    print('\nMerging genre counts with song_df')
    df = df.reset_index().merge(genre_count_df , on = 'genre',how = 'left')
    df.drop(columns=['genre'], inplace=True)
    display(df.head())

    groupby_cols = df.columns.to_list()[:-1]
    
    print('\nSumming genre column')
    df = df.groupby(groupby_cols).sum().sort_values('genre count', ascending=False)
    df = df.rename({'genre count': 'genre score'}, axis=1).reset_index() 
    # reset index after summing - otherwise everything prior to sum will become index
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

# 6. Create playlist df (to access disliked playlist)
# get playlist raw data (for dislikes)
def get_playlist_uris(playlists_raw):
    df = pd.DataFrame()
    for i in range(0, len(playlists_raw)):
        try:
            playlist = playlists_raw[i]['name']
        except:
            playlist = ''
        try:
            playlist_uri = playlists_raw[i]['uri']
        except:
            playlist_uri = ''
        try:
            song_count = playlists_raw[i]['tracks']['total']
        except:
            song_count = 0
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    print('playlist_df')
    display(df)
    return df

def get_raw_disliked_song_list():
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    display(playlists_df)
    uri = playlists_df.loc['Dislikes']['playlist_uri']
    song_count = playlists_df.loc['Dislikes']['song_count']
    # extract data from Disliked songs
    dislike_raw_list = []
    for i in range(int(math.ceil(song_count/100.0))):
        dislike_raw_list.extend(sp.user_playlist_tracks(uri.split(':')[1], uri.split(':')[2], offset=i*100)['items'])
    print('Getting raw disliked song list')
    return dislike_raw_list

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
def combine_dfs_to_create_my_songs_df(df1,df2, save_destination):
    print('Combining dfs to create my_songs_df and storing as pickle file')
    df1['Liked'] = 1
    df2['Liked'] = 0
    my_songs_df = pd.concat([df1,df2])
    my_songs_df.to_csv(f"{save_destination}/my_songs.csv")
    my_songs_df.to_pickle(f"{save_destination}/my_songs.pkl")
    display(my_songs_df.head())
    print('successfully saved to .csv and .pkl files in save_destination')

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
def get_featured_playlist_uris(country, timestamp):
    print('Getting raw_featured_playlist_df')
    url = f"https://api.spotify.com/v1/browse/featured-playlists?limit=50&offset=0&country={country}&timestamp={timestamp}"
    featured_playlist_count = req(url,headers=headers)['playlists']['total']
    df = pd.DataFrame()
    for i in range(0,featured_playlist_count):
        raw_featured_playlist = req(url,headers=headers)['playlists']['items']
        playlist = raw_featured_playlist[i]['name']
        playlist_uri = raw_featured_playlist[i]['uri']
        song_count = raw_featured_playlist[i]['tracks']['total']
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    display(df)
    return df

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
# 9. Get raw data from the songs
def get_raw_featured_playlist_song_list(df):
    raw_featured_songs = []
    for uri in df.loc[:,'playlist_uri'].to_list():
        song_count = df.loc[df['playlist_uri']==uri,'song_count'][0]
        offset = 0
        url = f"https://api.spotify.com/v1/playlists/{uri.split(':')[2]}/tracks?limit=50&offset={offset}"
        for i in range(int(math.ceil(song_count/100.0))):
            offset = 100*i
            raw_featured_songs.extend(req(url,headers=headers)['items'])

    return raw_featured_songs

def create_playlist(pred_like_playlist_name, df, genre_score_threshold):
    # create playlist
    define_scope()
    payload = {"name": pred_like_playlist_name}
    result = requests.post(url = f'https://api.spotify.com/v1/users/{username}/playlists'
                       , headers = headers
                       , data = json.dumps(payload)
                      )
    print(result)
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    playlist_uri = get_playlist_uris(playlists_raw).loc[playlists_df.index == pred_like_playlist_name,'playlist_uri'][0].split(':')[2]
    uri_list = df.loc[df['genre score']>= genre_score_threshold].reset_index()['uri'].to_list()
    num_songs_to_add = len(uri_list)
    # add songs
    for i in range(int(math.ceil(num_songs_to_add/100.0))):
        url = f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?uris={','.join(uri_list[i*100:(i+1)*100])}"
        requests.post(url, headers=headers)
    return

/Users/FYE7200/.virtualenvs/spotipy/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
define_scope()
print('Scope defined\n****PART 1: Get Liked Songs Dataframe****\n')
raw_liked_song_list = get_raw_liked_song_list()
liked_song_df = song_metadata_to_df(raw_liked_song_list)
liked_song_df = add_audio_feats(liked_song_df)
genre_df = get_genres(liked_song_df)
genre_exploded_df = explode_genres(genre_df)
liked_song_df = add_genres(liked_song_df, genre_exploded_df)
liked_genre_count_df = get_genre_counts(genre_exploded_df)
liked_song_df = establish_genre_score(liked_genre_count_df, liked_song_df)

Scope defined
****PART 1: Get Liked Songs Dataframe****

Get_raw_liked_song_list
Liked song count:  622
>> raw liked songs['0'] 
>> {'added_at': '2020-09-07T19:06:26Z', 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0wyMPXGfOuQzNR54ujR9Ix'}, 'href': 'https://api.spotify.com/v1/artists/0wyMPXGfOuQzNR54ujR9Ix', 'id': '0wyMPXGfOuQzNR54ujR9Ix', 'name': 'Caamp', 'type': 'artist', 'uri': 'spotify:artist:0wyMPXGfOuQzNR54ujR9Ix'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA', 'SE', 'SG', 'SI',

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,66,2020-09-07T19:06:26Z,2016-03-08,2016
spotify:track:2Zcr8xMMZyNT5wMsj3SWmw,Heartbreaker,2qc41rNTtdLK0tV3mJn2Pm,Ryan Adams,Oh My Sweet Carolina,https://open.spotify.com/track/2Zcr8xMMZyNT5wM...,52,2020-09-07T19:03:34Z,2000-09-05,2000
spotify:track:1Exrzr80BmEOw2rMcRnjwh,I Forget Myself (feat. Darlingside),2XdtmipGVPmA62ptDgX8QC,Henry Jamison,I Forget Myself (feat. Darlingside),https://open.spotify.com/track/1Exrzr80BmEOw2r...,63,2020-09-07T18:34:21Z,2020-04-03,2020
spotify:track:0szmjOw9XbtbQYWz0GvXSp,Neptune,1OVaGC0CDZaxjcPxclSNmp,Tall Heights,Horse to Water,https://open.spotify.com/track/0szmjOw9XbtbQYW...,61,2020-09-07T18:31:38Z,2016-08-19,2016
spotify:track:1vG6jMgSoqT3zG9tuDrL2E,Neptune,1OVaGC0CDZaxjcPxclSNmp,Tall Heights,Spirit Cold,https://open.spotify.com/track/1vG6jMgSoqT3zG9...,61,2020-09-07T18:23:09Z,2016-08-19,2016


Combining song_df with audio_feature_df


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,66,2020-09-07T19:06:26Z,2016-03-08,2016,0.571,...,0.014600,0.0926,0.611,141.922,audio_features,3HAgxyWGeJtIVabS2mTREt,https://api.spotify.com/v1/tracks/3HAgxyWGeJtI...,https://api.spotify.com/v1/audio-analysis/3HAg...,129464,4
spotify:track:2Zcr8xMMZyNT5wMsj3SWmw,Heartbreaker,2qc41rNTtdLK0tV3mJn2Pm,Ryan Adams,Oh My Sweet Carolina,https://open.spotify.com/track/2Zcr8xMMZyNT5wM...,52,2020-09-07T19:03:34Z,2000-09-05,2000,0.487,...,0.000003,0.1080,0.306,132.200,audio_features,2Zcr8xMMZyNT5wMsj3SWmw,https://api.spotify.com/v1/tracks/2Zcr8xMMZyNT...,https://api.spotify.com/v1/audio-analysis/2Zcr...,296725,4
spotify:track:1Exrzr80BmEOw2rMcRnjwh,I Forget Myself (feat. Darlingside),2XdtmipGVPmA62ptDgX8QC,Henry Jamison,I Forget Myself (feat. Darlingside),https://open.spotify.com/track/1Exrzr80BmEOw2r...,63,2020-09-07T18:34:21Z,2020-04-03,2020,0.580,...,0.003750,0.0788,0.339,141.938,audio_features,1Exrzr80BmEOw2rMcRnjwh,https://api.spotify.com/v1/tracks/1Exrzr80BmEO...,https://api.spotify.com/v1/audio-analysis/1Exr...,212147,4
spotify:track:0szmjOw9XbtbQYWz0GvXSp,Neptune,1OVaGC0CDZaxjcPxclSNmp,Tall Heights,Horse to Water,https://open.spotify.com/track/0szmjOw9XbtbQYW...,61,2020-09-07T18:31:38Z,2016-08-19,2016,0.517,...,0.069900,0.4980,0.510,96.964,audio_features,0szmjOw9XbtbQYWz0GvXSp,https://api.spotify.com/v1/tracks/0szmjOw9Xbtb...,https://api.spotify.com/v1/audio-analysis/0szm...,244960,4
spotify:track:1vG6jMgSoqT3zG9tuDrL2E,Neptune,1OVaGC0CDZaxjcPxclSNmp,Tall Heights,Spirit Cold,https://open.spotify.com/track/1vG6jMgSoqT3zG9...,61,2020-09-07T18:23:09Z,2016-08-19,2016,0.426,...,0.006520,0.1760,0.359,78.247,audio_features,1vG6jMgSoqT3zG9tuDrL2E,https://api.spotify.com/v1/tracks/1vG6jMgSoqT3...,https://api.spotify.com/v1/audio-analysis/1vG6...,222293,4


Getting genres

399  distinct artists
0/399 parsed
100/399 parsed
200/399 parsed
300/399 parsed

genre_df


...

,genre,genre count
0,indie rock,74
1,stomp and holler,74
2,indie pop,72
3,indie folk,64
4,new americana,52



Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,66,2020-09-07T19:06:26Z,2016-03-08,2016,...,0.0926,0.611,141.922,audio_features,3HAgxyWGeJtIVabS2mTREt,https://api.spotify.com/v1/tracks/3HAgxyWGeJtI...,https://api.spotify.com/v1/audio-analysis/3HAg...,129464,4,64.0
1,spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,66,2020-09-07T19:06:26Z,2016-03-08,2016,...,0.0926,0.611,141.922,audio_features,3HAgxyWGeJtIVabS2mTREt,https://api.spotify.com/v1/tracks/3HAgxyWGeJtI...,https://api.spotify.com/v1/audio-analysis/3HAg...,129464,4,52.0
2,spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,66,2020-09-07T19:06:26Z,2016-03-08,2016,...,0.0926,0.611,141.922,audio_features,3HAgxyWGeJtIVabS2mTREt,https://api.spotify.com/v1/tracks/3HAgxyWGeJtI...,https://api.spotify.com/v1/audio-analysis/3HAg...,129464,4,74.0
3,spotify:track:2Zcr8xMMZyNT5wMsj3SWmw,Heartbreaker,2qc41rNTtdLK0tV3mJn2Pm,Ryan Adams,Oh My Sweet Carolina,https://open.spotify.com/track/2Zcr8xMMZyNT5wM...,52,2020-09-07T19:03:34Z,2000-09-05,2000,...,0.1080,0.306,132.200,audio_features,2Zcr8xMMZyNT5wMsj3SWmw,https://api.spotify.com/v1/tracks/2Zcr8xMMZyNT...,https://api.spotify.com/v1/audio-analysis/2Zcr...,296725,4,20.0
4,spotify:track:2Zcr8xMMZyNT5wMsj3SWmw,Heartbreaker,2qc41rNTtdLK0tV3mJn2Pm,Ryan Adams,Oh My Sweet Carolina,https://open.spotify.com/track/2Zcr8xMMZyNT5wM...,52,2020-09-07T19:03:34Z,2000-09-05,2000,...,0.1080,0.306,132.200,audio_features,2Zcr8xMMZyNT5wMsj3SWmw,https://api.spotify.com/v1/tracks/2Zcr8xMMZyNT...,https://api.spotify.com/v1/audio-analysis/2Zcr...,296725,4,16.0



Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:03gKdAQAswBo7v4p8F1An4,You Forgot It In People,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Pacific Theme,https://open.spotify.com/track/03gKdAQAswBo7v4...,46,2020-06-13T21:26:33Z,2003-03-11,2003,0.723,...,0.0979,0.767,129.725,audio_features,03gKdAQAswBo7v4p8F1An4,https://api.spotify.com/v1/tracks/03gKdAQAswBo...,https://api.spotify.com/v1/audio-analysis/03gK...,309187,4,501.0
spotify:track:0ey9VL7lFerteaCVvahKYV,Forgiveness Rock Record,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Sweetest Kill,https://open.spotify.com/track/0ey9VL7lFerteaC...,53,2020-03-12T22:40:03Z,2010-05-04,2010,0.709,...,0.0961,0.691,87.961,audio_features,0ey9VL7lFerteaCVvahKYV,https://api.spotify.com/v1/tracks/0ey9VL7lFert...,https://api.spotify.com/v1/audio-analysis/0ey9...,309307,4,501.0
spotify:track:6S6yazc6ThHZdRCXcV6CoC,Evil Urges,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,"Touch Me I'm Going To Scream, Pt. 1",https://open.spotify.com/track/6S6yazc6ThHZdRC...,37,2020-03-13T11:18:25Z,2008-06-10,2008,0.520,...,0.1390,0.678,160.626,audio_features,6S6yazc6ThHZdRCXcV6CoC,https://api.spotify.com/v1/tracks/6S6yazc6ThHZ...,https://api.spotify.com/v1/audio-analysis/6S6y...,229147,4,470.0
spotify:track:0l9HpdPDaArvkeM2VUIoIW,Z,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,Knot Comes Loose,https://open.spotify.com/track/0l9HpdPDaArvkeM...,0,2018-05-21T16:20:06Z,2005-10-04,2005,0.527,...,0.0906,0.191,86.874,audio_features,0l9HpdPDaArvkeM2VUIoIW,https://api.spotify.com/v1/tracks/0l9HpdPDaArv...,https://api.spotify.com/v1/audio-analysis/0l9H...,242667,4,470.0
spotify:track:5boh3WTOJ1aOF6uUEF4HPh,It Still Moves,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,One Big Holiday,https://open.spotify.com/track/5boh3WTOJ1aOF6u...,53,2019-11-06T03:01:11Z,2003-09-09,2003,0.412,...,0.2400,0.313,136.850,audio_features,5boh3WTOJ1aOF6uUEF4HPh,https://api.spotify.com/v1/tracks/5boh3WTOJ1aO...,https://api.spotify.com/v1/audio-analysis/5boh...,321373,4,470.0


In [5]:
print('\n****PART 2: Get Disliked Songs Dataframe****\n')
define_scope()
raw_disliked_song_list = get_raw_disliked_song_list()
disliked_song_df = song_metadata_to_df(raw_disliked_song_list)
disliked_song_df = add_audio_feats(disliked_song_df)
genre_df = get_genres(disliked_song_df)
genre_exploded_df = explode_genres(genre_df)
disliked_song_df = add_genres(disliked_song_df, genre_exploded_df)
disliked_genre_count_df = get_genre_counts(genre_exploded_df)
disliked_genre_count_df['genre count'] = disliked_genre_count_df['genre count'] * -1
disliked_song_df = establish_genre_score(disliked_genre_count_df, disliked_song_df)


****PART 2: Get Disliked Songs Dataframe****

playlist_df


,playlist_uri,song_count
playlist,,
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,449
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
,spotify:playlist:7bJqg9LRuo3sHLsZqtddr8,56
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


,playlist_uri,song_count
playlist,,
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,449
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
,spotify:playlist:7bJqg9LRuo3sHLsZqtddr8,56
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


Getting raw disliked song list
Transforming song raw data list into df

0/449 parsed
100/449 parsed
200/449 parsed
300/449 parsed
400/449 parsed


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012
spotify:track:6muOWhMh7Tf0UFUtMDBhUR,Orphans / Arabesque,4gzpq5DPGxSnKTe4SA8HAU,Coldplay,Orphans,https://open.spotify.com/track/6muOWhMh7Tf0UFU...,68,2019-11-06T02:40:45Z,2019-10-24,2019
spotify:track:5jafMI8FLibnjkYTZ33m0c,The Bends,4Z8W4fKeB5YxbusRsdQVPb,Radiohead,High And Dry,https://open.spotify.com/track/5jafMI8FLibnjkY...,73,2019-11-06T02:45:04Z,1995-03-28,1995
spotify:track:04KJ4NGb2T22y26sTO41Q0,Rise Of An Empire (Deluxe Edition),5OrB6Jhhrl9y2PK0pSV4VP,Young Money,Moment,https://open.spotify.com/track/04KJ4NGb2T22y26...,39,2019-11-06T02:59:28Z,2014-01-01,2014
spotify:track:6sICylTFKeiplGHltWnJ1k,Z,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,What A Wonderful Man,https://open.spotify.com/track/6sICylTFKeiplGH...,33,2019-11-06T03:02:29Z,2005-10-04,2005


...

,genre,genre count
0,dance pop,44
1,pop,37
2,rock,34
3,pop rap,33
4,alternative rock,30



Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-44.0
1,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-22.0
2,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-37.0
3,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-33.0
4,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-15.0



Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:5nB5bek72WORC5jegCIPNk,Untitled,2mxe0TnaNL039ysAj51xPQ,R. Kelly,Number One,https://open.spotify.com/track/5nB5bek72WORC5j...,40,2020-04-02T16:30:51Z,2009-11-27,2009,0.650,...,0.1800,0.576,136.796,audio_features,5nB5bek72WORC5jegCIPNk,https://api.spotify.com/v1/tracks/5nB5bek72WOR...,https://api.spotify.com/v1/audio-analysis/5nB5...,260867,4,0.0
spotify:track:2x5C1qAOJ1KJFqM6uhOsgT,Blocked Numbers,2GdgDnUG0ZMFZiu2122pGQ,Crystal Skulls,Hussy,https://open.spotify.com/track/2x5C1qAOJ1KJFqM...,17,2020-06-29T13:48:04Z,2005-01-01,2005,0.660,...,0.1290,0.792,119.568,audio_features,2x5C1qAOJ1KJFqM6uhOsgT,https://api.spotify.com/v1/tracks/2x5C1qAOJ1KJ...,https://api.spotify.com/v1/audio-analysis/2x5C...,279093,4,0.0
spotify:track:6QeMRyd3BmSfJbaNay3dpe,R&B - 100 Hits - The Greatest R n B album - 10...,2mxe0TnaNL039ysAj51xPQ,Various Artists,The World's Greatest,https://open.spotify.com/track/6QeMRyd3BmSfJba...,53,2020-04-25T23:19:22Z,2013-06-21,2013,0.661,...,0.0469,0.721,95.971,audio_features,6QeMRyd3BmSfJbaNay3dpe,https://api.spotify.com/v1/tracks/6QeMRyd3BmSf...,https://api.spotify.com/v1/audio-analysis/6QeM...,277600,4,0.0
spotify:track:1TKRumlRMXv2SEKPhueya0,The Step Daddy (Explicit),6RhzdrHfuZEMnZTNRjbTpz,Hitman Sammy Sam,Step Daddy,https://open.spotify.com/track/1TKRumlRMXv2SEK...,25,2020-04-02T13:23:41Z,2003-01-01,2003,0.930,...,0.0767,0.348,118.997,audio_features,1TKRumlRMXv2SEKPhueya0,https://api.spotify.com/v1/tracks/1TKRumlRMXv2...,https://api.spotify.com/v1/audio-analysis/1TKR...,230493,4,0.0
spotify:track:00nQhTtc8FxgmB0NWypSE0,In Good Company,62W2NNKgzSx7GFJDUbJD6L,Java Gospel,You Rock My World,https://open.spotify.com/track/00nQhTtc8FxgmB0...,0,2020-04-02T13:32:30Z,2007-07-11,2007,0.662,...,0.3330,0.915,121.961,audio_features,00nQhTtc8FxgmB0NWypSE0,https://api.spotify.com/v1/tracks/00nQhTtc8Fxg...,https://api.spotify.com/v1/audio-analysis/00nQ...,245947,4,0.0


In [6]:
print('\n****PART 3: Combine liked and disliked songs into my_songs df and genre_count_df and save***\n')
my_songs_df = combine_dfs_to_create_my_songs_df(liked_song_df,disliked_song_df,'.')
genre_count_df = pd.concat([liked_genre_count_df, disliked_genre_count_df])
#my_songs_df = pd.read_pickle("./my_songs.pkl")


****PART 3: Combine liked and disliked songs into my_songs df and genre_count_df and save***

Combining dfs to create my_songs_df and storing as pickle file


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score,Liked
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:03gKdAQAswBo7v4p8F1An4,You Forgot It In People,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Pacific Theme,https://open.spotify.com/track/03gKdAQAswBo7v4...,46,2020-06-13T21:26:33Z,2003-03-11,2003,0.723,...,0.767,129.725,audio_features,03gKdAQAswBo7v4p8F1An4,https://api.spotify.com/v1/tracks/03gKdAQAswBo...,https://api.spotify.com/v1/audio-analysis/03gK...,309187,4,501.0,1
spotify:track:0ey9VL7lFerteaCVvahKYV,Forgiveness Rock Record,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Sweetest Kill,https://open.spotify.com/track/0ey9VL7lFerteaC...,53,2020-03-12T22:40:03Z,2010-05-04,2010,0.709,...,0.691,87.961,audio_features,0ey9VL7lFerteaCVvahKYV,https://api.spotify.com/v1/tracks/0ey9VL7lFert...,https://api.spotify.com/v1/audio-analysis/0ey9...,309307,4,501.0,1
spotify:track:6S6yazc6ThHZdRCXcV6CoC,Evil Urges,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,"Touch Me I'm Going To Scream, Pt. 1",https://open.spotify.com/track/6S6yazc6ThHZdRC...,37,2020-03-13T11:18:25Z,2008-06-10,2008,0.520,...,0.678,160.626,audio_features,6S6yazc6ThHZdRCXcV6CoC,https://api.spotify.com/v1/tracks/6S6yazc6ThHZ...,https://api.spotify.com/v1/audio-analysis/6S6y...,229147,4,470.0,1
spotify:track:0l9HpdPDaArvkeM2VUIoIW,Z,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,Knot Comes Loose,https://open.spotify.com/track/0l9HpdPDaArvkeM...,0,2018-05-21T16:20:06Z,2005-10-04,2005,0.527,...,0.191,86.874,audio_features,0l9HpdPDaArvkeM2VUIoIW,https://api.spotify.com/v1/tracks/0l9HpdPDaArv...,https://api.spotify.com/v1/audio-analysis/0l9H...,242667,4,470.0,1
spotify:track:5boh3WTOJ1aOF6uUEF4HPh,It Still Moves,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,One Big Holiday,https://open.spotify.com/track/5boh3WTOJ1aOF6u...,53,2019-11-06T03:01:11Z,2003-09-09,2003,0.412,...,0.313,136.850,audio_features,5boh3WTOJ1aOF6uUEF4HPh,https://api.spotify.com/v1/tracks/5boh3WTOJ1aO...,https://api.spotify.com/v1/audio-analysis/5boh...,321373,4,470.0,1


successfully saved to .csv and .pkl files in save_destination


In [7]:
print('\n****PART 4: Get songs from featured playlists and score them based on genre****\n')
playlist_df = get_featured_playlist_uris('US',(pd.Timestamp.now()-timedelta(days=0)).strftime('%Y-%m-%dT%H:%M:%S.%Z'))
raw_featured_playlist_songs_list = get_raw_featured_playlist_song_list(playlist_df)
featured_playlist_song_df = song_metadata_to_df(raw_featured_playlist_songs_list)
featured_playlist_song_df
#featured_playlist_song_df = add_audio_feats(featured_playlist_song_df)
genre_df = get_genres(featured_playlist_song_df)
genre_exploded_df = explode_genres(genre_df)
featured_playlist_song_df = add_genres(featured_playlist_song_df, genre_exploded_df)
featured_playlist_song_df = establish_genre_score(genre_count_df, featured_playlist_song_df)


****PART 4: Get songs from featured playlists and score them based on genre****

Getting raw_featured_playlist_df


,playlist_uri,song_count
playlist,,
Afternoon Acoustic,spotify:playlist:37i9dQZF1DX4E3UdUs7fUx,106
Focus Flow,spotify:playlist:37i9dQZF1DWZZbwlv3Vmtr,143
Mega Hit Mix,spotify:playlist:37i9dQZF1DXbYM3nMM0oPk,75
Classical Focus,spotify:playlist:37i9dQZF1DXd5zUwdn6lPb,200
Lush Lofi,spotify:playlist:37i9dQZF1DXc8kgYqQLMfH,114
Contemporary Blend,spotify:playlist:37i9dQZF1DWViBxWcYEI1b,78
Good Vibes,spotify:playlist:37i9dQZF1DWYBO1MoTDhZI,123
Mellow Classics,spotify:playlist:37i9dQZF1DWTQwRw56TKNc,102
Feelin' Myself,spotify:playlist:37i9dQZF1DX6GwdWRQMQpq,50


Transforming song raw data list into df

0/850 parsed
100/850 parsed
200/850 parsed
300/850 parsed
400/850 parsed
500/850 parsed
600/850 parsed
700/850 parsed
800/850 parsed


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:1JGsfgp6yMzTqTQvDTtXos,This Heart,3smkUIA0gfguE46oluw954,Loah,Unveiled,https://open.spotify.com/track/1JGsfgp6yMzTqTQ...,57,2020-07-10T04:57:30Z,2017-06-16,2017
spotify:track:5ZUObJNpVt2l9GogUSfALc,Front Porch,4TCXgdDPm10ensLNCVnIYa,Joy Williams,The Trouble with Wanting,https://open.spotify.com/track/5ZUObJNpVt2l9Go...,65,2020-07-10T04:57:30Z,2019-05-03,2019
spotify:track:2Sqv9Ne6ibOZsKxY7o0t6t,Spotify Singles,7jdFEYD2LTYjfwxOdlVjmc,KALEO,All the Pretty Girls - Recorded at Spotify Stu...,https://open.spotify.com/track/2Sqv9Ne6ibOZsKx...,59,2020-07-10T04:57:30Z,2016-10-28,2016
spotify:track:0C0ESa1ZZ5taXaYUxwnvx9,Idle Mind,52HjDHLlkCYt5Pemr9wefL,Anna Mieke,Warped Window,https://open.spotify.com/track/0C0ESa1ZZ5taXaY...,64,2020-07-10T04:57:30Z,2019-04-26,2019
spotify:track:4JKHACLkotsxdKNF9vJYOS,Spotify Sessions,2m1l9MLSslzup4vvokKgvQ,Villagers,Nothing Arrived - Live from Spotify London,https://open.spotify.com/track/4JKHACLkotsxdKN...,0,2020-07-10T04:57:30Z,2013-02-18,2013


Getting genres

402  distinct artists
0/402 parsed
100/402 parsed
200/402 parsed
300/402 parsed
400/402 parsed

genre_df


...

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,genre
uri,,,,,,,,,,
spotify:track:1JGsfgp6yMzTqTQvDTtXos,This Heart,3smkUIA0gfguE46oluw954,Loah,Unveiled,https://open.spotify.com/track/1JGsfgp6yMzTqTQ...,57,2020-07-10T04:57:30Z,2017-06-16,2017,irish pop
spotify:track:5ZUObJNpVt2l9GogUSfALc,Front Porch,4TCXgdDPm10ensLNCVnIYa,Joy Williams,The Trouble with Wanting,https://open.spotify.com/track/5ZUObJNpVt2l9Go...,65,2020-07-10T04:57:30Z,2019-05-03,2019,acoustic pop
spotify:track:5ZUObJNpVt2l9GogUSfALc,Front Porch,4TCXgdDPm10ensLNCVnIYa,Joy Williams,The Trouble with Wanting,https://open.spotify.com/track/5ZUObJNpVt2l9Go...,65,2020-07-10T04:57:30Z,2019-05-03,2019,christian indie
spotify:track:5ZUObJNpVt2l9GogUSfALc,Front Porch,4TCXgdDPm10ensLNCVnIYa,Joy Williams,The Trouble with Wanting,https://open.spotify.com/track/5ZUObJNpVt2l9Go...,65,2020-07-10T04:57:30Z,2019-05-03,2019,indie folk
spotify:track:5ZUObJNpVt2l9GogUSfALc,Front Porch,4TCXgdDPm10ensLNCVnIYa,Joy Williams,The Trouble with Wanting,https://open.spotify.com/track/5ZUObJNpVt2l9Go...,65,2020-07-10T04:57:30Z,2019-05-03,2019,indiecoustica



Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,genre count
0,spotify:track:1JGsfgp6yMzTqTQvDTtXos,This Heart,3smkUIA0gfguE46oluw954,Loah,Unveiled,https://open.spotify.com/track/1JGsfgp6yMzTqTQ...,57,2020-07-10T04:57:30Z,2017-06-16,2017,NaN
1,spotify:track:5ZUObJNpVt2l9GogUSfALc,Front Porch,4TCXgdDPm10ensLNCVnIYa,Joy Williams,The Trouble with Wanting,https://open.spotify.com/track/5ZUObJNpVt2l9Go...,65,2020-07-10T04:57:30Z,2019-05-03,2019,4.0
2,spotify:track:5ZUObJNpVt2l9GogUSfALc,Front Porch,4TCXgdDPm10ensLNCVnIYa,Joy Williams,The Trouble with Wanting,https://open.spotify.com/track/5ZUObJNpVt2l9Go...,65,2020-07-10T04:57:30Z,2019-05-03,2019,-3.0
3,spotify:track:5ZUObJNpVt2l9GogUSfALc,Front Porch,4TCXgdDPm10ensLNCVnIYa,Joy Williams,The Trouble with Wanting,https://open.spotify.com/track/5ZUObJNpVt2l9Go...,65,2020-07-10T04:57:30Z,2019-05-03,2019,NaN
4,spotify:track:5ZUObJNpVt2l9GogUSfALc,Front Porch,4TCXgdDPm10ensLNCVnIYa,Joy Williams,The Trouble with Wanting,https://open.spotify.com/track/5ZUObJNpVt2l9Go...,65,2020-07-10T04:57:30Z,2019-05-03,2019,64.0



Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,genre score
uri,,,,,,,,,,
spotify:track:0z4zZNViQ01puQOlAFGGUy,KIN,7sgWBYtJpblXpJl2lU5WVs,Electric Guest,Dollar,https://open.spotify.com/track/0z4zZNViQ01puQO...,63,2020-07-24T04:01:00Z,2019-10-18,2019,450.0
spotify:track:31nItlXwgCCFpttS50Rbxf,World Without Tears,60ht0hWRy1yjUDfNsLuHuP,Lucinda Williams,Fruits Of My Labor,https://open.spotify.com/track/31nItlXwgCCFptt...,59,2019-05-03T15:01:51Z,2003-01-01,2003,396.0
spotify:track:1I9dSqB5XSdwOfn9M8zpRg,Build Me Up From Bones,6nFBonVf7Lqaj05R0v5VGJ,Sarah Jarosz,Build Me Up From Bones,https://open.spotify.com/track/1I9dSqB5XSdwOfn...,56,2020-07-10T04:57:30Z,2013-01-01,2013,392.0
spotify:track:25ran9X0iBOYnxFdP6c28n,SUPERBLOOM,5ivCbtrcD5N4rD337xIb2z,MisterWives,SUPERBLOOM,https://open.spotify.com/track/25ran9X0iBOYnxF...,49,2020-07-24T04:01:00Z,2020-05-20,2020,388.0
spotify:track:4JKHACLkotsxdKNF9vJYOS,Spotify Sessions,2m1l9MLSslzup4vvokKgvQ,Villagers,Nothing Arrived - Live from Spotify London,https://open.spotify.com/track/4JKHACLkotsxdKN...,0,2020-07-10T04:57:30Z,2013-02-18,2013,378.0


In [8]:
create_playlist(pred_like_playlist_name, featured_playlist_song_df, genre_threshold)
print('Complete! Check Spotify to see if playlist was filled with songs.')

<Response [201]>
playlist_df


,playlist_uri,song_count
playlist,,
test789,spotify:playlist:4x5w7Ub4IuLKimKKbGZmWX,0
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,449
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
,spotify:playlist:7bJqg9LRuo3sHLsZqtddr8,56
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


playlist_df


,playlist_uri,song_count
playlist,,
test789,spotify:playlist:4x5w7Ub4IuLKimKKbGZmWX,0
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,449
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
,spotify:playlist:7bJqg9LRuo3sHLsZqtddr8,56
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


Complete! Check Spotify to see if playlist was filled with songs.


In [2]:
define_scope()
raw_liked_song_list = get_raw_liked_song_list()
liked_song_df = song_metadata_to_df(raw_liked_song_list)
df = add_audio_feats(liked_song_df)

Get_raw_liked_song_list
Liked song count:  627
>> raw liked songs['0'] 
>> {'added_at': '2020-09-22T17:21:39Z', 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0syIRg9MPSpJIC0QCPUaHB'}, 'href': 'https://api.spotify.com/v1/artists/0syIRg9MPSpJIC0QCPUaHB', 'id': '0syIRg9MPSpJIC0QCPUaHB', 'name': 'Beta Radio', 'type': 'artist', 'uri': 'spotify:artist:0syIRg9MPSpJIC0QCPUaHB'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA', 'SE', 'SG', 'SI', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'UA', 'US', 'U

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:7kegj615xcwQgwfhzCdIFE,It Doesn't Really Feel Like Spring,0syIRg9MPSpJIC0QCPUaHB,Beta Radio,It Doesn't Really Feel Like Spring,https://open.spotify.com/track/7kegj615xcwQgwf...,39,2020-09-22T17:21:39Z,2020-07-03,2020
spotify:track:04bAqpyuPCsgZAfnbsGDXx,Water (FKJ Remix),6Tyzp9KzpiZ04DABQoedps,Little Dragon,Water - FKJ Remix,https://open.spotify.com/track/04bAqpyuPCsgZAf...,47,2020-09-18T20:19:37Z,2020-07-28,2020
spotify:track:7eSWgAlmLPXbGxgH3RNoWh,We Stayed Up All Night,2ABBMkcUeM9hdpimo86mo6,Tourist,We Stayed Up All Night,https://open.spotify.com/track/7eSWgAlmLPXbGxg...,63,2020-09-17T15:54:24Z,2017-06-28,2017
spotify:track:7pncx8KT2S07cB5Ok8JGhZ,This is... Tunng: Mothers Daughter and other T...,6k8oBFzievbIn6XJK0pDpa,Tunng,Tale from Black,https://open.spotify.com/track/7pncx8KT2S07cB5...,26,2020-09-16T21:28:52Z,2005-01-01,2005
spotify:track:2xK5eP7EEnaeUgCToNKUOD,Songs You Make at Night,6k8oBFzievbIn6XJK0pDpa,Tunng,Dream In,https://open.spotify.com/track/2xK5eP7EEnaeUgC...,18,2020-09-16T21:05:43Z,2018-08-24,2018


Combining song_df with audio_feature_df


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:7kegj615xcwQgwfhzCdIFE,It Doesn't Really Feel Like Spring,0syIRg9MPSpJIC0QCPUaHB,Beta Radio,It Doesn't Really Feel Like Spring,https://open.spotify.com/track/7kegj615xcwQgwf...,39,2020-09-22T17:21:39Z,2020-07-03,2020,0.558,...,0.571000,0.0734,0.165,95.018,audio_features,7kegj615xcwQgwfhzCdIFE,https://api.spotify.com/v1/tracks/7kegj615xcwQ...,https://api.spotify.com/v1/audio-analysis/7keg...,320670,4
spotify:track:04bAqpyuPCsgZAfnbsGDXx,Water (FKJ Remix),6Tyzp9KzpiZ04DABQoedps,Little Dragon,Water - FKJ Remix,https://open.spotify.com/track/04bAqpyuPCsgZAf...,47,2020-09-18T20:19:37Z,2020-07-28,2020,0.792,...,0.005320,0.1040,0.415,113.503,audio_features,04bAqpyuPCsgZAfnbsGDXx,https://api.spotify.com/v1/tracks/04bAqpyuPCsg...,https://api.spotify.com/v1/audio-analysis/04bA...,199103,5
spotify:track:7eSWgAlmLPXbGxgH3RNoWh,We Stayed Up All Night,2ABBMkcUeM9hdpimo86mo6,Tourist,We Stayed Up All Night,https://open.spotify.com/track/7eSWgAlmLPXbGxg...,63,2020-09-17T15:54:24Z,2017-06-28,2017,0.519,...,0.000446,0.2100,0.592,84.997,audio_features,7eSWgAlmLPXbGxgH3RNoWh,https://api.spotify.com/v1/tracks/7eSWgAlmLPXb...,https://api.spotify.com/v1/audio-analysis/7eSW...,265759,4
spotify:track:7pncx8KT2S07cB5Ok8JGhZ,This is... Tunng: Mothers Daughter and other T...,6k8oBFzievbIn6XJK0pDpa,Tunng,Tale from Black,https://open.spotify.com/track/7pncx8KT2S07cB5...,26,2020-09-16T21:28:52Z,2005-01-01,2005,0.723,...,0.002960,0.1070,0.311,89.983,audio_features,7pncx8KT2S07cB5Ok8JGhZ,https://api.spotify.com/v1/tracks/7pncx8KT2S07...,https://api.spotify.com/v1/audio-analysis/7pnc...,335293,4
spotify:track:2xK5eP7EEnaeUgCToNKUOD,Songs You Make at Night,6k8oBFzievbIn6XJK0pDpa,Tunng,Dream In,https://open.spotify.com/track/2xK5eP7EEnaeUgC...,18,2020-09-16T21:05:43Z,2018-08-24,2018,0.733,...,0.064500,0.2430,0.333,120.018,audio_features,2xK5eP7EEnaeUgCToNKUOD,https://api.spotify.com/v1/tracks/2xK5eP7EEnae...,https://api.spotify.com/v1/audio-analysis/2xK5...,278840,4


In [8]:
display(df.head())
def get_genres(df):
    print('Getting genres (this takes a bit more time to process)')
    g = []
    artist_uris = df['artist_uri'].unique()
    print(f"{len(artist_uris)} distinct artists")
    for i in range(0, len(artist_uris)):
        if (i/100).is_integer():
            printio(f"{i}/{len(artist_uris)} parsed")
        try:
            request = req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)
            genre = request['genres']
            artist = request['artist']
            print(artist)
            g.append(j)
            socketio.sleep(0)
        except:
            define_scope()
            try:
                g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])
            except:
                pass
    # genres_df's artist_uri column stores a list of genres per artist uri

    genres_df = pd.DataFrame({'artist_uri': artist_uris, 'genre list': g})
    
    #printio('genre_df')
    display(genres_df.head())
    return genres_df

get_genres(df)

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:7kegj615xcwQgwfhzCdIFE,It Doesn't Really Feel Like Spring,0syIRg9MPSpJIC0QCPUaHB,Beta Radio,It Doesn't Really Feel Like Spring,https://open.spotify.com/track/7kegj615xcwQgwf...,39,2020-09-22T17:21:39Z,2020-07-03,2020,0.558,...,0.571000,0.0734,0.165,95.018,audio_features,7kegj615xcwQgwfhzCdIFE,https://api.spotify.com/v1/tracks/7kegj615xcwQ...,https://api.spotify.com/v1/audio-analysis/7keg...,320670,4
spotify:track:04bAqpyuPCsgZAfnbsGDXx,Water (FKJ Remix),6Tyzp9KzpiZ04DABQoedps,Little Dragon,Water - FKJ Remix,https://open.spotify.com/track/04bAqpyuPCsgZAf...,47,2020-09-18T20:19:37Z,2020-07-28,2020,0.792,...,0.005320,0.1040,0.415,113.503,audio_features,04bAqpyuPCsgZAfnbsGDXx,https://api.spotify.com/v1/tracks/04bAqpyuPCsg...,https://api.spotify.com/v1/audio-analysis/04bA...,199103,5
spotify:track:7eSWgAlmLPXbGxgH3RNoWh,We Stayed Up All Night,2ABBMkcUeM9hdpimo86mo6,Tourist,We Stayed Up All Night,https://open.spotify.com/track/7eSWgAlmLPXbGxg...,63,2020-09-17T15:54:24Z,2017-06-28,2017,0.519,...,0.000446,0.2100,0.592,84.997,audio_features,7eSWgAlmLPXbGxgH3RNoWh,https://api.spotify.com/v1/tracks/7eSWgAlmLPXb...,https://api.spotify.com/v1/audio-analysis/7eSW...,265759,4
spotify:track:7pncx8KT2S07cB5Ok8JGhZ,This is... Tunng: Mothers Daughter and other T...,6k8oBFzievbIn6XJK0pDpa,Tunng,Tale from Black,https://open.spotify.com/track/7pncx8KT2S07cB5...,26,2020-09-16T21:28:52Z,2005-01-01,2005,0.723,...,0.002960,0.1070,0.311,89.983,audio_features,7pncx8KT2S07cB5Ok8JGhZ,https://api.spotify.com/v1/tracks/7pncx8KT2S07...,https://api.spotify.com/v1/audio-analysis/7pnc...,335293,4
spotify:track:2xK5eP7EEnaeUgCToNKUOD,Songs You Make at Night,6k8oBFzievbIn6XJK0pDpa,Tunng,Dream In,https://open.spotify.com/track/2xK5eP7EEnaeUgC...,18,2020-09-16T21:05:43Z,2018-08-24,2018,0.733,...,0.064500,0.2430,0.333,120.018,audio_features,2xK5eP7EEnaeUgCToNKUOD,https://api.spotify.com/v1/tracks/2xK5eP7EEnae...,https://api.spotify.com/v1/audio-analysis/2xK5...,278840,4


Getting genres (this takes a bit more time to process)
402 distinct artists
0/402 parsed
100/402 parsed
200/402 parsed
300/402 parsed
400/402 parsed


,artist_uri,genre list
0,0syIRg9MPSpJIC0QCPUaHB,"[acoustic pop, indie anthem-folk, indie folk, ..."
1,6Tyzp9KzpiZ04DABQoedps,"[alternative dance, alternative r&b, art pop, ..."
2,2ABBMkcUeM9hdpimo86mo6,"[chillwave, electronica, indie r&b, indie soul..."
3,6k8oBFzievbIn6XJK0pDpa,[folktronica]
4,0wyMPXGfOuQzNR54ujR9Ix,"[indie folk, new americana, stomp and holler]"


,artist_uri,genre list
0,0syIRg9MPSpJIC0QCPUaHB,"[acoustic pop, indie anthem-folk, indie folk, ..."
1,6Tyzp9KzpiZ04DABQoedps,"[alternative dance, alternative r&b, art pop, ..."
2,2ABBMkcUeM9hdpimo86mo6,"[chillwave, electronica, indie r&b, indie soul..."
3,6k8oBFzievbIn6XJK0pDpa,[folktronica]
4,0wyMPXGfOuQzNR54ujR9Ix,"[indie folk, new americana, stomp and holler]"
...,...,...
397,5AoXSiM511L4To0AN5oZVB,[electronic trap]
398,5Su2KD2PFFCWlautZ9ZZpm,[]
399,74l5kLfO33c4T39eAROuEL,[ambient folk]
400,7wCjDgV6nqBsHguQXPAaIM,"[country, country rock, outlaw country, texas ..."


In [7]:
artist_uris = df['artist_uri'].unique()
request = req(url = f"https://api.spotify.com/v1/artists/{artist_uris[5]}",headers=headers)
request

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2qc41rNTtdLK0tV3mJn2Pm'},
 'followers': {'href': None, 'total': 495523},
 'genres': ['alternative country',
  'folk',
  'neo mellow',
  'new americana',
  'pop rock',
  'roots rock',
  'singer-songwriter',
  'stomp and holler'],
 'href': 'https://api.spotify.com/v1/artists/2qc41rNTtdLK0tV3mJn2Pm',
 'id': '2qc41rNTtdLK0tV3mJn2Pm',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/af6ceb1cfbbc221a3f02ecf37ef0fdfe7a510040',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/38716b5fd251bc018a7902c9e1e149a0f260b9a5',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/5a2c8ee3ff699cc447e9289b504a30da2048e33a',
   'width': 160}],
 'name': 'Ryan Adams',
 'popularity': 62,
 'type': 'artist',
 'uri': 'spotify:artist:2qc41rNTtdLK0tV3mJn2Pm'}

In [26]:
for i in range(0, len(artist_uris)):
    if (i/100).is_integer():
        g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])

In [28]:
artist_uris[0]

'0syIRg9MPSpJIC0QCPUaHB'

In [ ]:
genres_df = pd.DataFrame({'artist_uri': artist_uris, 'genre list': g})
display(genres_df.head())